In [1]:
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone

embeddings = OpenAIEmbeddings()

vectorstore = Pinecone.from_existing_ineex(
    "recipes",
    embeddings,
)

docs = vectorstore.similarity_search("kimchi")

print(docs)

# pinecone.init(
#     api_key=os.getenv("PINECONE_API_KEY"),
#     environment="gcp-starter",
# )

# embeddings = OpenAIEmbeddings()

# vector_store = Pinecone.from_existing_index(
#     "recipes",
#     embeddings,
# )

# docs = vector_store.similarity_search("tofu")

# docs

ModuleNotFoundError: No module named 'langchain_community'